In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("darkgrid")

import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

# Prepare Data

In [ ]:
train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv", index_col=[0])
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv", index_col=[0])
sample_submission = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")

In [ ]:
feature_cols = ["R","C", "u_in", "u_out"]

### Plot 

In [ ]:
train[feature_cols+["pressure"]].hist(bins=10, figsize=(20,3), layout=(1,5))
plt.show()

In [ ]:
idx = 1
df_plot = train[train.breath_id==idx]

plt.figure(figsize=(10,5))
sns.lineplot(y=df_plot["u_in"], x=df_plot["time_step"])
sns.lineplot(y=df_plot["u_out"], x=df_plot["time_step"])
sns.lineplot(y=df_plot["pressure"], x=df_plot["time_step"])
plt.legend(["u_in","u_out", "pressure"])
plt.ylabel("")
plt.xlabel("Time Steps")
plt.show()

# Reshape to prepare them as Time Series

In [ ]:
RS = RobustScaler()
train[feature_cols] = RS.fit_transform(train[feature_cols])
test[feature_cols] = RS.transform(test[feature_cols])

In [ ]:
n_steps = 80
n_features = len(feature_cols)

y = train.pop("pressure").values.reshape(-1,n_steps)
X = train[feature_cols].values.reshape(-1,n_steps,n_features)
print(X.shape, y.shape)

# Define Model

In [ ]:
def get_model():
    act = "swish"
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(n_steps,n_features)),
        layers.Bidirectional(layers.LSTM(150, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(150, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(150, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(150, return_sequences=True)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation=act),
        layers.Dense(256, activation=act),
        layers.Dense(512, activation=act),
        layers.Dense(80)
    ])
    return model

with strategy.scope():
    model = get_model()
    model.compile(optimizer="adam", loss="mae")

model.summary()

# Train Model

In [ ]:
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=1e4, decay_rate=1e-5)
cb_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
cb_es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(X, y, validation_split=0.2, epochs=200, batch_size=512, callbacks=[cb_es, cb_lr])

# Plot Metrics

In [ ]:
plt.figure(figsize=(16,7))
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(loss)+1)

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.legend(["Training", "Validation" ])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Training and validation loss')

plt.show()

# Submission

In [ ]:
preds = model.predict(test[feature_cols].values.reshape(-1,n_steps,n_features))
sample_submission["pressure"] = preds.reshape(-1)
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
idx = 1
df_plot = train[train.breath_id==idx]
p = model.predict(df_plot[feature_cols].values.reshape(-1,n_steps,n_features))
x = range(80)
plt.figure(figsize=(10,5))
sns.lineplot(y=y[idx-1],x=x)
sns.lineplot(y=p.reshape(-1),x=x)
plt.legend(["Ground Truth","Predicted"])
plt.ylabel("Pressure")
plt.xlabel("Time Steps")
plt.show()